In [ ]:
import pandas as pd
import numpy as np

# Import Plotting Libararies
import seaborn as sns
import matplotlib.pyplot as plt
import re
import gensim
# Import Data Preprocessing Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
from tensorflow.keras.optimizers import Adam, Optimizer, SGD
import tensorflow.keras.initializers as Init
from tensorflow.keras import regularizers

In [ ]:
project = pd.read_excel('Dataset Here')

In [ ]:
project.head()

In [ ]:

project.isnull().sum()

In [ ]:
project.dropna()

Comments  Sentiments
0      tum jaisi aurat ko to mar jana chahie har jaga...           0
1      Meri hamesha se ye khwahish rahi hai ke me log...           1
2      tughe to me ghar me ghus kar maroo ga tgh jais...           0
3      Jab hum apki baatein sunte hai to dil me apki ...           1
4      saali randi ki aulaad gashti aurat nangi hi ku...           0
...                                                  ...         ...
11239  bhai is canbee lifestyle vlogs wale ko roast k...           0
11240        bhai bhut chutiyape ke video lagi ye to               0
11241  aa backchodi baba iman phek k maru tujhe mothe...           0
11242  tum jase chutia ki bajase hi india ka nam dubt...           0
11243  bhai bsdk qna rkh or mera question ye h ki kab...           0

[11244 rows x 2 columns]

In [ ]:
project.isnull().sum()

In [ ]:
project[project['Sentiments'] == 'Neative'].index

Int64Index([], dtype='int64')

In [ ]:
project.info()

In [ ]:
project.shape

In [ ]:
def cleaner(word):
    word = re.sub(r'\#\.', '', word)
    word = re.sub(r'\n', '', word)
    word = re.sub(r',', '', word)
    word = re.sub(r'\-', ' ', word)
    word = re.sub(r'\.', '', word)
    word = re.sub(r'\\', ' ', word)
    word = re.sub(r'\\x\.+', '', word)
    word = re.sub(r'\d', '', word)
    word = re.sub(r'^_.', '', word)
    word = re.sub(r'_', ' ', word)
    word = re.sub(r'^ ', '', word)
    word = re.sub(r' $', '', word)
    word = re.sub(r'\?', '', word)
    word = re.sub(r'é', '', word)
    word = re.sub(r'§', '', word)
    word = re.sub(r'¦', '', word)
    word = re.sub(r'æ', '', word)
    word = re.sub(r'\d+', '', word)
    word = re.sub('(.*?)\d+(.*?)', '', word)
    return word.lower()
def hashing(word):
    word = re.sub(r'ain$', r'ein', word)
    word = re.sub(r'ai', r'ae', word)
    word = re.sub(r'ay$', r'e', word)
    word = re.sub(r'ey$', r'e', word)
    word = re.sub(r'ie$', r'y', word)
    word = re.sub(r'^es', r'is', word)
    word = re.sub(r'a+', r'a', word)
    word = re.sub(r'j+', r'j', word)
    word = re.sub(r'd+', r'd', word)
    word = re.sub(r'u', r'o', word)
    word = re.sub(r'o+', r'o', word)
    word = re.sub(r'ee+', r'i', word)

    if not re.match(r'ar', word):
        word = re.sub(r'ar', r'r', word)
    word = re.sub(r'iy+', r'i', word)
    word = re.sub(r'ih+', r'eh', word)
    word = re.sub(r's+', r's', word)
    if re.search(r'[rst]y', 'word') and word[-1] != 'y':
        word = re.sub(r'y', r'i', word)
    if re.search(r'[bcdefghijklmnopqrtuvwxyz]i', word):
        word = re.sub(r'i$', r'y', word)
    if re.search(r'[acefghijlmnoqrstuvwxyz]h', word):
        word = re.sub(r'h', '', word)
    word = re.sub(r'k', r'q', word)
    return word

def array_cleaner(array):
    X = []
    for sentence in array:
        clean_sentence = ''
        words = sentence.split(' ')
        for word in words:
            clean_sentence = clean_sentence +' '+ cleaner(word)
        X.append(clean_sentence)
    return X

In [ ]:
def deleteUncharacter(texts):
    clean_text=[]
    for cur in texts:
        cur = re.sub('[^a-zA-Z]',' ',cur)
        cur=cur.lower()
        clean_text.append(cur)
    return clean_text

In [ ]:
# Remove stop words from text
from typing import FrozenSet

# Urdu Language Stop words list
STOP_WORDS: FrozenSet[str] = frozenset("""
  ai ayi hy hai main ki tha koi ko sy woh bhi aur wo yeh rha hota ho ga ka le lye kr kar lye liye hotay waisay gya gaya kch ab thy thay houn hain han to is hi jo kya thi se pe phr wala waisay us na ny hun rha raha rahay abi uski ne haan acha nai sent photo you kafi gai rhy kuch jata aye ya dono hoa aese de wohi jati jb krta lg rahi hui karna krna gi hova yehi jana jye chal mil tu hum par hay kis sb gy dain krny tou
""".split())


def remove_stopwords(text: str):
    return " ".join(word for word in text.split() if word not in STOP_WORDS)

In [ ]:
project['Comments'] = project['Comments'].apply(remove_stopwords)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(project['Comments'], project['Sentiments'], test_size = 0.10, random_state = 1, shuffle = True)

In [ ]:
word_2_vec = project.copy()

In [ ]:
import spacy
def tokenizer(str):
    nlp = spacy.blank('en')
    doc = nlp.tokenizer(str)
    return [i.text for i in doc]
word_2_vec["tokens"] = word_2_vec["Comments"].apply(tokenizer)

In [ ]:
model_word2vec = gensim.models.Word2Vec(sentences=word_2_vec["tokens"], size=128, window=2, workers=5, min_count = 1)

In [ ]:
VOCAB_SIZE = len(model_word2vec.wv.vocab)
DIMENSIONS = 128
MAX_LEN = max([len(x) for x in word_2_vec["tokens"]])

In [ ]:
VOCAB_SIZE, DIMENSIONS, MAX_LEN

In [ ]:
from keras.preprocessing.text import Tokenizer#to tokenize the data and made text sequences.
token = Tokenizer()
token.fit_on_texts(word_2_vec["tokens"])
encoded = token.texts_to_sequences(word_2_vec["tokens"])

In [ ]:
words2vec_matrix = np.zeros((VOCAB_SIZE+1,DIMENSIONS))
for word, index in token.word_index.items():
    try:
        words2vec_matrix[index] = model_word2vec.wv[word]
    except:
        print(index, word)

In [ ]:
import tensorflow as tf
train_vectors = tf.keras.preprocessing.sequence.pad_sequences(encoded,padding='post',dtype=int)

In [ ]:
train_label = word_2_vec.Sentiments

In [ ]:
(train_sentences,test_sentences, train_tags, test_tags) = train_test_split(train_vectors, train_label, test_size=0.1, shuffle = True)

In [ ]:
train_sentences

In [ ]:
#model preparation
max_features =14226
embedding_dim =128
sequence_length = MAX_LEN

LSTM_model = tf.keras.Sequential()
LSTM_model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                    embeddings_initializer = Init.Constant(words2vec_matrix),embeddings_regularizer = regularizers.l2(0.005)))

LSTM_model.add(tf.keras.layers.Dropout(0.1))

LSTM_model.add(tf.keras.layers.LSTM(embedding_dim,dropout=0.1, recurrent_dropout=0.2,return_sequences=True,\
                                                             kernel_regularizer=regularizers.l2(0.005),\
                                                             bias_regularizer=regularizers.l2(0.005)))

LSTM_model.add(tf.keras.layers.Flatten())

LSTM_model.add(tf.keras.layers.Dense(512, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
LSTM_model.add(tf.keras.layers.Dropout(0.1))

LSTM_model.add(tf.keras.layers.Dense(8, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
LSTM_model.add(tf.keras.layers.Dropout(0.1))


LSTM_model.add(tf.keras.layers.Dense(1,activation='sigmoid'))




LSTM_model.summary()
LSTM_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(1e-3),metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
                                                                                                                                                                                                                lstm = Models.Sequential()

lstm.add(Layers.Embedding(VOCAB_SIZE+1,DIMENSIONS,
                          embeddings_initializer = Init.Constant(words2vec_matrix),
                          input_length=MAX_LEN, trainable=True ))

lstm.add(Layers.Bidirectional(Layers.LSTM(256, activation='tanh')))

lstm.add(Layers.Dense(128, activation='tanh'))
lstm.add(Layers.Dropout(0.3))

lstm.add(Layers.Dense(64, activation='tanh'))
lstm.add(Layers.Dropout(0.3))

lstm.add(Layers.Dense(1, activation='sigmoid'))

lstm.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping


lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es_callback = EarlyStopping(monitor='val_loss', patience=3)
LSTM_NET = lstm.fit(train_sentences, train_tags, epochs=10, validation_split=0.2, callbacks=[es_callback], shuffle=False)

In [ ]:
print(classification_report(lstm.predict(test_sentences).round(), test_tags))

In [ ]:
plt.plot(LSTM_NET.history['accuracy'])
plt.plot(LSTM_NET.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model = Models.Sequential()

model.add(Layers.Embedding(VOCAB_SIZE+1,DIMENSIONS,
                           embeddings_initializer = Init.Constant(words2vec_matrix),
                           input_length=MAX_LEN, trainable=True ))

model.add(Layers.Conv1D(512, 5, activation="relu"))
model.add(Layers.MaxPooling1D(5))


model.add(Layers.Conv1D(128, 3, activation="relu"))
model.add(Layers.Dropout(0.1))
model.add(Layers.MaxPooling1D(3))

model.add(Layers.Conv1D(64, 1, activation="relu"))
model.add(Layers.Dropout(0.1))
model.add(Layers.MaxPooling1D(1))

model.add(Layers.Conv1D(32, 1, activation="relu"))
model.add(Layers.Dropout(0.1))

model.add(Layers.Flatten())

model.add(Layers.Dense(32, activation='relu', kernel_regularizer = regularizers.l2(1e-4)))
model.add(Layers.Dropout(0.1))

model.add(Layers.Dense(1,activation='sigmoid'))

model.summary()

model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
CNN = model.fit( train_sentences, train_tags, epochs=10, validation_split=0.10 )

In [ ]:
plt.plot(CNN.history['accuracy'])
plt.plot(CNN.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
print(classification_report(model.predict(test_sentences).round(), test_tags))